# LDA

In [27]:
# topic modelling Quran

In [45]:
# READ TEXT and CREATE DOCUMENT - sentences in list
# Uses the translation from Yousaf Ali
# Available at
# https://www.sacred-texts.com/isl/yaq/index.htm
fname = "//Quran/YousafAliQ.txt"
fobj = open(fname, "r")
doc_complete = [""]

for l in fobj:
    # print(l)
    l = l.strip()
    # if not l:
    if not l:  # because am and pm is used
        continue
        print("FOUND EMPTY or Hanging String")
    else:
        #print(l)
        if len(l)>1:
            doc_complete.append(l.strip())
        

print("Total documents ", len(doc_complete))
print (type(doc_complete))
print (doc_complete[:10])

Total documents  40485
<class 'list'>
['', 'Sura I.', 'Fatiha, or the Opening Chapter.', '1. In the name of God, Most Gracious,', 'Most Merciful.', '2. Praise be to God,', 'The Cherisher and Sustainer of the Worlds;', '3. Most Gracious, Most Merciful;', '4. Master of the Day of Judgment.', '5. Thee do we worship,']


In [55]:
# change to data frame for better handling
import pandas as pd
docs = pd.DataFrame(doc_complete)
#print(docs[:10])
print(len(docs))
print(docs[0][5])

40485
2. Praise be to God,


In [60]:
# fix imports
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
import numpy as np
np.random.seed(2018)
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\usman\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [65]:
# preprocess
# clean and then stem
def lemmatize_stemming(text):
    return WordNetLemmatizer().lemmatize(text, pos='v')
def preprocess(text):
    result = []
    for token in gensim.utils.simple_preprocess(text):
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 2:
            result.append(lemmatize_stemming(token))
    return result

In [66]:
doc_sample = docs[0][5]
print('original document: ')
words = []
for word in doc_sample.split(' '):
    words.append(word)
print(words)
print('\n\n tokenized and lemmatized document: ')
print(preprocess(doc_sample))

original document: 
['2.', 'Praise', 'be', 'to', 'God,']


 tokenized and lemmatized document: 
['praise', 'god']


In [74]:
# do preprocess on frame
pro_docs = docs[0].map(preprocess)
pro_docs[:10]

0                                []
1                            [sura]
2           [fatiha, open, chapter]
3                   [god, gracious]
4                        [merciful]
5                     [praise, god]
6    [cherisher, sustainer, worlds]
7              [gracious, merciful]
8           [master, day, judgment]
9                   [thee, worship]
Name: 0, dtype: object

In [75]:
# create dictionary
qd = gensim.corpora.Dictionary(pro_docs)
#count = 0
#for k, v in dictionary.iteritems():
    #print(k, v)
    #count += 1
    #if count > 10:
    #    break

In [76]:
qd.filter_extremes(no_below=15, no_above=0.5)

In [82]:
bagOfwords = [dictionary.doc2bow(doc) for doc in pro_docs]
bagOfwords[5]

[(4, 1), (7, 1)]

In [83]:
# explanation
chwo = bagOfwords[5]
for i in range(len(chwo)):
    print("Word {} (\"{}\") appears {} time.".format(chwo[i][0], dictionary[chwo[i][0]], chwo[i][1]))

Word 4 ("god") appears 1 time.
Word 7 ("praise") appears 1 time.


In [88]:
# create the tfidf
from gensim import corpora, models

tfidf = models.TfidfModel(bagOfwords)
corpus_tfidf = tfidf[bagOfwords]
from pprint import pprint
count = 0
for doc in corpus_tfidf:
    pprint(doc)
    count+=1
    if count>5:
        break

[]
[(0, 1.0)]
[(1, 0.6486035766533425), (2, 0.6486035766533425), (3, 0.3982798020299091)]
[(4, 0.4237363322512761), (5, 0.9057855820944801)]
[(6, 1.0)]
[(4, 0.39497463122018167), (7, 0.9186920271192526)]


In [99]:
# create model
lda_model = gensim.models.LdaMulticore(bagOfwords, num_topics=5, id2word=dictionary, passes=5)

In [100]:
for idx, topic in lda_model.print_topics(-1):
    print('Topic: {} \n {}'.format(idx, topic))

Topic: 0 
 0.190*"god" + 0.044*"say" + 0.034*"believe" + 0.026*"section" + 0.023*"know" + 0.019*"turn" + 0.016*"fear" + 0.012*"gracious" + 0.011*"away" + 0.011*"truly"
Topic: 1 
 0.030*"people" + 0.021*"men" + 0.017*"create" + 0.015*"follow" + 0.014*"unbelievers" + 0.014*"power" + 0.013*"let" + 0.013*"hath" + 0.012*"like" + 0.012*"man"
Topic: 2 
 0.040*"thee" + 0.034*"send" + 0.029*"sign" + 0.023*"reject" + 0.020*"good" + 0.018*"faith" + 0.015*"verily" + 0.014*"knowledge" + 0.013*"deeds" + 0.012*"clear"
Topic: 3 
 0.051*"thou" + 0.039*"day" + 0.029*"shall" + 0.019*"things" + 0.017*"earth" + 0.017*"penalty" + 0.016*"bring" + 0.012*"reward" + 0.011*"forgive" + 0.010*"forth"
Topic: 4 
 0.070*"lord" + 0.035*"thy" + 0.031*"come" + 0.021*"truth" + 0.020*"give" + 0.018*"evil" + 0.017*"apostle" + 0.015*"merciful" + 0.014*"wrong" + 0.014*"heavens"
